In [1]:
import pandas as pd

# Load the datasets
crop_df = pd.read_csv("datasets\crop.csv")
imd_df = pd.read_csv("datasets\Sub_Division_IMD_2017.csv")

# ==============================================================================
# 1. Cleaning the Crop Production Data (crop_df)
# ==============================================================================

# 1.1 Handle missing 'Production' values by dropping rows
# Dropping is preferred over imputation for production quantity due to high variability.
rows_before = len(crop_df)
crop_df_cleaned = crop_df.dropna(subset=['Production']).copy()
rows_dropped = rows_before - len(crop_df_cleaned)
print(f"Removed {rows_dropped} rows with missing 'Production'.")

# 1.2 Ensure 'Production' is numeric (although it was float64, this step is good practice)
crop_df_cleaned['Production'] = pd.to_numeric(
    crop_df_cleaned['Production'], errors='coerce'
)
crop_df_cleaned.dropna(subset=['Production'], inplace=True)


# 1.3 Handle logical inconsistencies: Area > 0 but Production = 0
# These rows often represent crop failures or data entry errors and are removed for robust analysis.
inconsistent_rows = crop_df_cleaned[
    (crop_df_cleaned['Area'] > 0) & (crop_df_cleaned['Production'] == 0)
]
inconsistent_count = len(inconsistent_rows)

crop_df_cleaned = crop_df_cleaned[
    ~((crop_df_cleaned['Area'] > 0) & (crop_df_cleaned['Production'] == 0))
].copy()

print(f"Removed {inconsistent_count} rows with Area > 0 but Production = 0.")
print(f"Final size of cleaned crop data: {len(crop_df_cleaned)} rows.")

# ==============================================================================
# 2. Cleaning the Rainfall Data (imd_df)
# ==============================================================================

# 2.1 Impute missing values in rainfall columns
# Rainfall columns start from index 2 ('JAN') to the end.
rain_cols = imd_df.columns[2:]

# Use the median for imputation as it is robust to extreme rainfall outliers.
imd_df_cleaned = imd_df.copy()
for col in rain_cols:
    median_val = imd_df_cleaned[col].median()
    imd_df_cleaned[col].fillna(median_val, inplace=True)

print("\nMissing values in rainfall data imputed using the median for each column.")


# ==============================================================================
# 3. Save the cleaned datasets
# ==============================================================================
crop_df_cleaned.to_csv("crop_cleaned.csv", index=False)
imd_df_cleaned.to_csv("rainfall_cleaned.csv", index=False)

print("\n--- Cleaning Complete ---")
print("Cleaned crop data saved to 'crop_cleaned.csv'")
print("Cleaned rainfall data saved to 'rainfall_cleaned.csv'")

<>:4: SyntaxWarning: invalid escape sequence '\c'
<>:5: SyntaxWarning: invalid escape sequence '\S'
<>:4: SyntaxWarning: invalid escape sequence '\c'
<>:5: SyntaxWarning: invalid escape sequence '\S'
C:\Users\dhany\AppData\Local\Temp\ipykernel_17432\350215933.py:4: SyntaxWarning: invalid escape sequence '\c'
  crop_df = pd.read_csv("datasets\crop.csv")
C:\Users\dhany\AppData\Local\Temp\ipykernel_17432\350215933.py:5: SyntaxWarning: invalid escape sequence '\S'
  imd_df = pd.read_csv("datasets\Sub_Division_IMD_2017.csv")


Removed 3730 rows with missing 'Production'.
Removed 3523 rows with Area > 0 but Production = 0.
Final size of cleaned crop data: 238838 rows.

Missing values in rainfall data imputed using the median for each column.


C:\Users\dhany\AppData\Local\Temp\ipykernel_17432\350215933.py:51: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  imd_df_cleaned[col].fillna(median_val, inplace=True)
C:\Users\dhany\AppData\Local\Temp\ipykernel_17432\350215933.py:51: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For


--- Cleaning Complete ---
Cleaned crop data saved to 'crop_cleaned.csv'
Cleaned rainfall data saved to 'rainfall_cleaned.csv'


In [3]:
import pandas as pd
import numpy as np

# Load the cleaned datasets
crop_df = pd.read_csv("datasets/crop_cleaned.csv")
rainfall_df = pd.read_csv("datasets/rainfall_cleaned.csv")

# ==============================================================================
# 1. Define the Mapping for State_Name to SUBDIVISION
# ==============================================================================

# This dictionary maps a State/UT name (from crop data) to the list of
# meteorological sub-divisions (from rainfall data) it covers.
state_to_subdivision_map = {
    'Andaman and Nicobar Islands': ['Andaman & Nicobar Islands'],
    'Andhra Pradesh': ['Coastal Andhra Pradesh', 'Rayalseema'],
    'Arunachal Pradesh': ['Arunachal Pradesh'],
    'Assam': ['Assam & Meghalaya'],
    'Bihar': ['Bihar'],
    'Chandigarh': ['Haryana Delhi & Chandigarh'],
    'Chhattisgarh': ['Chhattisgarh'],
    'Dadra and Nagar Haveli': ['Gujarat Region', 'Saurashtra & Kutch'],
    'Goa': ['Konkan & Goa'],
    'Gujarat': ['Gujarat Region', 'Saurashtra & Kutch'],
    'Haryana': ['Haryana Delhi & Chandigarh'],
    'Himachal Pradesh': ['Himachal Pradesh'],
    'Jammu and Kashmir': ['Jammu & Kashmir'],
    'Jharkhand': ['Jharkhand'],
    'Karnataka': ['Coastal Karnataka', 'North Interior Karnataka', 'South Interior Karnataka'],
    'Kerala': ['Kerala'],
    'Madhya Pradesh': ['East Madhya Pradesh', 'West Madhya Pradesh'],
    'Maharashtra': ['Konkan & Goa', 'Madhya Maharashtra', 'Matathwada', 'Vidarbha'],
    'Manipur': ['Naga Mani Mizo Tripura'],
    'Meghalaya': ['Assam & Meghalaya'],
    'Mizoram': ['Naga Mani Mizo Tripura'],
    'Nagaland': ['Naga Mani Mizo Tripura'],
    'Odisha': ['Orissa'],
    'Puducherry': ['Tamil Nadu'],
    'Punjab': ['Punjab'],
    'Rajasthan': ['East Rajasthan', 'West Rajasthan'],
    'Sikkim': ['Sub Himalayan West Bengal & Sikkim'],
    'Tamil Nadu': ['Tamil Nadu'],
    'Telangana': ['Telangana'],
    'Tripura': ['Naga Mani Mizo Tripura'],
    'Uttar Pradesh': ['East Uttar Pradesh', 'West Uttar Pradesh'],
    'Uttarakhand': ['Uttarakhand'],
    'West Bengal': ['Gangetic West Bengal', 'Sub Himalayan West Bengal & Sikkim']
}


# ==============================================================================
# 2. Pre-processing the Crop Data for Merging
# ==============================================================================

# 2.1 Clean State Names and create a list of subdivisions for each crop row
crop_df['State_Name'] = crop_df['State_Name'].str.strip()
crop_df['SUBDIVISION_List'] = crop_df['State_Name'].map(state_to_subdivision_map)

# 2.2 Explode the DataFrame and rename columns for merging
# This step creates multiple rows for states that span multiple rainfall sub-divisions.
crop_df_expanded = crop_df.explode('SUBDIVISION_List').rename(
    columns={'SUBDIVISION_List': 'SUBDIVISION', 'Crop_Year': 'YEAR'}
)

# ==============================================================================
# 3. Merging the Datasets
# ==============================================================================

# 3.1 Perform the inner merge on both YEAR and SUBDIVISION
integrated_df = pd.merge(
    crop_df_expanded,
    rainfall_df,
    on=['YEAR', 'SUBDIVISION'],
    how='inner'
)

# 3.2 Calculate Yield (Production per Area)
integrated_df['Yield (Production/Area)'] = integrated_df['Production'] / integrated_df['Area']

# Save the final integrated dataset
integrated_df.to_csv("crop_rainfall_integrated.csv", index=False)

In [1]:
import pandas as pd
import sqlite3

# --- Configuration ---
csv_file = 'datasets/crop_rainfall_integrated_cleaned.csv'
db_file = 'samarth_agri_climate.db'
table_name = 'integrated_data'

# --- 1. Load the CSV data into a Pandas DataFrame ---
try:
    df = pd.read_csv(csv_file)
    print(f"Successfully loaded {csv_file}. DataFrame shape: {df.shape}")
except FileNotFoundError:
    print(f"Error: The file {csv_file} was not found.")
    exit()

# --- 2. Create a database connection ---
# This creates the SQLite database file if it doesn't exist.
conn = sqlite3.connect(db_file)
print(f"Successfully connected to the database: {db_file}")

# --- 3. Use to_sql() to push the DataFrame content into the DB ---
# if_exists='replace': Drops the table if it exists and creates a new one.
# index=False: Prevents the DataFrame's index from becoming a column.
df.to_sql(table_name, conn, if_exists='replace', index=False)
print(f"Successfully inserted data into table: {table_name}")

# --- 4. Verification (Reading data back from the DB) ---
query = f"SELECT * FROM {table_name} LIMIT 5;"
df_from_db = pd.read_sql_query(query, conn)
print("\nVerification: Data read back from the database:")
print(df_from_db)

# --- 5. Close the connection ---
conn.close()
print("\nDatabase connection closed.")

Successfully loaded datasets/crop_rainfall_integrated_cleaned.csv. DataFrame shape: (412295, 27)
Successfully connected to the database: samarth_agri_climate.db
Successfully inserted data into table: integrated_data

Verification: Data read back from the database:
                         state  district  year       season  \
0  andaman and nicobar islands  NICOBARS  2000  Kharif        
1  andaman and nicobar islands  NICOBARS  2000  Kharif        
2  andaman and nicobar islands  NICOBARS  2000  Kharif        
3  andaman and nicobar islands  NICOBARS  2000  Whole Year    
4  andaman and nicobar islands  NICOBARS  2000  Whole Year    

                  crop  area_ha  production_tonnes  \
0             Arecanut   1254.0             2000.0   
1  Other Kharif pulses      2.0                1.0   
2                 Rice    102.0              321.0   
3               Banana    176.0              641.0   
4            Cashewnut    720.0              165.0   

                   subdivision 